In [127]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import time


In [106]:
# Initialize Pinecone client (serverless)
api_key = "pcsk_2t1R3i_CTfQkahGG7XPoN5G7aBEpTQ4FgbPjGmxwSkSrMb48FpXKyhoNK8HPJftEKGyf2E"
index_name = "medical-chatbot"

In [107]:
pinecone_client = Pinecone(api_key=api_key)

In [108]:
if index_name not in pinecone_client.list_indexes().names():
    pinecone_client.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


In [109]:
index = pinecone_client.Index(index_name)

# %%

In [110]:
# Load PDFs
def load_pdf(directory_path):
    loader = DirectoryLoader(directory_path, glob="*.pdf", loader_cls=PyPDFLoader)
    return loader.load()

extracted_data = load_pdf(r"C:\Users\DELL\medical-chatbot--Llama2\Data")


In [111]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    return text_splitter.split_documents(extracted_data)

In [112]:
text_chunks = text_split(extracted_data)
print("Number of chunks:", len(text_chunks))

Number of chunks: 10734


In [113]:
def download_hugging_face_embeddings():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embeddings = download_hugging_face_embeddings()


In [114]:
query_result = embeddings.embed_query("Hello world")
print("Embedding vector length:", len(query_result))


Embedding vector length: 384


In [115]:
docsearch = LangchainPinecone.from_texts(
    [chunk.page_content for chunk in text_chunks],
    embedding=embeddings,
    index_name=index_name
)

In [116]:
docsearch = LangchainPinecone.from_existing_index(index_name, embeddings)

# Define your query
query = "What are Allergies?"

# Perform similarity search
docs = docsearch.similarity_search(query, k=3)

# Print the results
print("Top 3 Results:\n")
for i, doc in enumerate(docs, 1):
    print(f"Result {i}:\n{doc.page_content}\n{'-'*80}")

Top 3 Results:

Result 1:
30514712]ALLERGIC RHINITISE S S E N T I A L S  O F  D I A G N O S I S»Clear rhinorrhea, sneezing, tearing, eye irritation,and pruritus.»Associated symptoms include cough, broncho-spasm, and eczematous dermatitis.»Environmental allergen exposure in the presenceof allergen-specific IgE.
--------------------------------------------------------------------------------
Result 2:
Dust, household mites, air pollu-tion, and pet dander may produce year-round symptoms,termed “perennial rhinitis. ”»Clinical FindingsThe symptoms of “hay fever” are similar to those of viralrhinitis but are usually persistent and may show seasonalvariation. Nasal symptoms are often accompanied by eyeirritation, pruritus, conjunctival erythema, and excessivetearing. Many patients have a strong family history of atopyor allergy.The clinician should be careful to distinguish allergicrhinitis from
--------------------------------------------------------------------------------
Result 3:
lower r

In [117]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [118]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [139]:
# Wrap it in a dictionary for RetrievalQA
chain_type_kwargs = {"prompt": prompt}

In [135]:
llm = CTransformers(
    model=r"C:\Users\DELL\medical-chatbot--Llama2\model\llama-2-7b-chat.ggmlv3.q4_0.bin",  # Replace with your actual local path
    model_type="llama",
    config={
        "max_new_tokens": 256,
        "temperature": 0.7,
        "threads": 4
    }
)


In [140]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs
)

In [142]:
while True:
    user_input = input("Input Prompt (or type 'exit' to quit): ")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chat.")
        break
    try:
        result = qa({"query": user_input})
        print("Response:", result["result"])
    except Exception as e:
        print("Error during response generation:", str(e))

C:\Users\DELL\AppData\Local\Temp\ipykernel_10752\2103010240.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa({"query": user_input})


Response: Allergies occur when your immune system overreacts to a foreign substance, such as dust, pollen, or pet dander, and mistakenly identifies it as harmful. This can lead to a range of symptoms including sneezing, congestion, runny nose, and itchy eyes. In severe cases, allergies can cause anaphylaxis, a life-threatening reaction that requires immediate medical attention.


Number of tokens (539) exceeded maximum context length (512).


KeyboardInterrupt: 